In [ ]:
# parameters
PROJECT_ID = "nice-proposal-467718-q6"
REGION = "us-west1"
BRONZE_PATH = "gs://meu-bucket-premier/bronze/"
SILVER_PATH = "gs://meu-bucket-premier/silver/"
GOLD_PATH = "gs://meu-bucket-premier/gold/"
RUN_TS = None


In [54]:
import pandas as pd

# Caminho no GCS
csv_path = "gs://meu-bucket-premier/bronze/epl/matches_2014_2020/matches_2014_2020.csv"

# Ler CSV direto do GCS
df = pd.read_csv(csv_path, storage_options={"token": "cloud"})

# Mostrar as primeiras linhas
print(df.shape)
df.head()

(5320, 43)


,Unnamed: 0,Home Team,Away Team,Score,Half Time Score,Match Excitement,Home Team Rating,Away Team Rating,Home Team Possession %,Away Team Possession %,...,Away Team Yellow Cards,Away Team Second Yellow Cards,Away Team Red Cards,Home Team Goals Scored,Away Team Goals Scored,Home Team Goals Conceeded,Away Team Goals Conceeded,year,_ingestion_ts,_dataset
0,0.0,MAN UTD,SWANSEA,1-2,0-1,5.9,5.6,7.6,60,40,...,4.0,0.0,0.0,1.0,2.0,2.0,1.0,2014.0,2025-08-13T14:39:37+00:00,matches_2014_2020
1,1.0,WEST BROM,SUNDERLAND,2-2,1-1,7.3,6.5,7.4,58,42,...,1.0,0.0,0.0,2.0,2.0,2.0,2.0,2014.0,2025-08-13T14:39:37+00:00,matches_2014_2020
2,2.0,LEICESTER CITY,EVERTON,2-2,1-2,7.0,6.5,6.3,37,63,...,1.0,0.0,0.0,2.0,2.0,2.0,2.0,2014.0,2025-08-13T14:39:37+00:00,matches_2014_2020
3,3.0,WEST HAM,TOTTENHAM,0-1,0-0,4.8,5.9,6.4,47,53,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,2014.0,2025-08-13T14:39:37+00:00,matches_2014_2020
4,4.0,QPR,HULL CITY,0-1,0-0,3.8,5.7,6.6,51,49,...,2.0,0.0,0.0,0.0,1.0,1.0,0.0,2014.0,2025-08-13T14:39:37+00:00,matches_2014_2020


In [55]:
df = df.drop("Unnamed: 0", axis= 1)
df = df.drop("_ingestion_ts", axis= 1)

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5320 entries, 0 to 5319
Data columns (total 41 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Home Team                      5320 non-null   object 
 1   Away Team                      5320 non-null   object 
 2   Score                          5320 non-null   object 
 3   Half Time Score                5320 non-null   object 
 4   Match Excitement               5320 non-null   float64
 5   Home Team Rating               5320 non-null   float64
 6   Away Team Rating               5320 non-null   float64
 7   Home Team Possession %         5320 non-null   int64  
 8   Away Team Possession %         5320 non-null   int64  
 9   Home Team Off Target Shots     5320 non-null   float64
 10  Home Team On Target Shots      5320 non-null   float64
 11  Home Team Total Shots          5320 non-null   float64
 12  Home Team Blocked Shots        5320 non-null   f

In [57]:
df.isnull().sum()

Home Team                           0
Away Team                           0
Score                               0
Half Time Score                     0
Match Excitement                    0
Home Team Rating                    0
Away Team Rating                    0
Home Team Possession %              0
Away Team Possession %              0
Home Team Off Target Shots          0
Home Team On Target Shots           0
Home Team Total Shots               0
Home Team Blocked Shots             0
Home Team Corners                   0
Home Team Throw Ins                 0
Home Team Pass Success %            0
Home Team Aerials Won               0
Home Team Clearances                0
Home Team Fouls                     0
Home Team Yellow Cards              0
Home Team Second Yellow Cards       0
Home Team Red Cards                 0
Away Team Off Target Shots          0
Away Team On Target Shots           0
Away Team Total Shots               0
Away Team Blocked Shots             0
Away Team Co

#### Tomada de decisão

- **Problema identificado**: Os campos *Home Team Goals Scored*, *Away Team Goals Scored*, *Home Team Goals Conceded* e *Away Team Goals Conceded* continham valores nulos.
- **Ação tomada**: Substituí os valores nulos por 0.
- **Justificativa**: No contexto do futebol, a ausência de informação nesses campos significa que a equipe não marcou ou não sofreu gols na partida.

In [58]:
df.fillna(0, inplace = True)

In [62]:
df.year.unique()

array([2014, 2015, 2016, 2017, 2018, 2019, 2020])

In [60]:
df["year"] = df["year"].astype(int)

In [61]:
df = df[df["year"] != 0]

In [63]:
# Caminho na camada Silver
silver_path = "gs://meu-bucket-premier/silver/epl/matches_2014_2020/matches_2014_2020_silver.csv"

# Salvando o DataFrame tratado
df.to_csv(
    silver_path,
    index=False,
    storage_options={"token": "cloud"}
)

print(f"✅ Arquivo salvo com sucesso na Silver: {silver_path}")


✅ Arquivo salvo com sucesso na Silver: gs://meu-bucket-premier/silver/epl/matches_2014_2020/matches_2014_2020_silver.csv


In [53]:
!gsutil ls gs://meu-bucket-premier/silver/epl/

gs://meu-bucket-premier/silver/epl/matches_2014_2020/
